In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import datadict
import functions
import devices
import plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.preprocessing import MinMaxScaler

In [6]:
from intakedata import Naming
n = Naming()
n.raw_devices


['Organic PV',
 'Silicon tandem',
 'Monocrystalline',
 'Polycrystalline',
 'CdTe',
 'CIGS']

In [161]:
data = datadict.DataFile("data_new_PV.json")

True

# Check for Missing

In [ ]:

for device_type in ['Organic PV', 'Silicon tandem', 'Monocrystalline', 'Polycrystalline', 'CdTe', 'CIGS']:
    data.module_type_selected = device_type
    data.get_layers()
    # create cell layer values
    print("------------------")
    print(device_type)
    for layer in data.layers:
        print(f"  - {layer}")
        for layer_option in data.data_dict[device_type][layer].keys():
            
            options_keys = data.data_dict[device_type][layer][layer_option]['Location'].keys()
            print(f"    - {layer_option}")
            for location_option in options_keys:
                n_keys = len(list(data.data_dict[device_type][layer][layer_option]['Location'][location_option].keys()))
                # if n_keys<18:
                print(f"      - {location_option}: {n_keys}")

# Compression Functions

## Example

In [41]:
# what the indicators
indicators = [
    'Quantity per m2',
    'energy resources: non-renewable, fossil',
    'acidification: terrestrial',
    'climate change',
    'ecotoxicity: freshwater',
    'eutrophication: freshwater',
    'eutrophication: marine',
    'human toxicity: carcinogenic',
    'human toxicity: non-carcinogenic',
    'ionising radiation',
    'land use',
    'ozone depletion',
    'particulate matter formation',
    'photochemical oxidant formation: human health',
    'photochemical oxidant formation: terrestrial ecosystems'
]

# what is the tco currently made of?
tco = {
    "Indium tin oxide": {
        "Location": {
            "RoW": {
                "Ecoinvent key": "c15a417acce2dc6d301385fc58ac0f5a",
                "Method LCIA": "Recipe 2016",
                "Unit": "kilogram",
                "Quantity per m2": 0.0129,
                "energy resources: non-renewable, fossil": 4.479164660830311,
                "acidification: terrestrial": 0.005725354615137292,
                "climate change": 0.39799539203094675,
                "ecotoxicity: freshwater": 0.5542311148814346,
                "eutrophication: freshwater": 0.0007272978509908413,
                "eutrophication: marine": 2.5730273724712597e-05,
                "human toxicity: carcinogenic": 0.12213193070191457,
                "human toxicity: non-carcinogenic": 11.901658235906527,
                "ionising radiation": 0.04955992999145971,
                "land use": 0.022397080984838755,
                "ozone depletion": 2.3373322165181446e-07,
                "particulate matter formation": 0.0020082643109328056,
                "photochemical oxidant formation: human health": 0.002004709377662678,
                "photochemical oxidant formation: terrestrial ecosystems": 0.0020771954096098035,
            },
            "RER": {
                "Ecoinvent key": "4925993ad37792748cd8578608765623",
                "Method LCIA": "Recipe 2016",
                "Unit": "kilogram",
                "Quantity per m2": 0.0129,
                "energy resources: non-renewable, fossil": 4.016767754012703,
                "acidification: terrestrial": 0.005560783404729452,
                "climate change": 0.35013322689786697,
                "ecotoxicity: freshwater": 0.553915658919175,
                "eutrophication: freshwater": 0.0007247991861945358,
                "eutrophication: marine": 2.5676291726771096e-05,
                "human toxicity: carcinogenic": 0.12116070295923964,
                "human toxicity: non-carcinogenic": 11.88618544563673,
                "ionising radiation": 0.06602606394199664,
                "land use": 0.022247448556872756,
                "ozone depletion": 2.239077644920298e-07,
                "particulate matter formation": 0.0018750668109078687,
                "photochemical oxidant formation: human health": 0.0018599852383655742,
                "photochemical oxidant formation: terrestrial ecosystems": 0.0019313626654658678,
            },
        }
    }
} 

# calculate the mean of the TCO from the two locations
tco_mean = dict(
    [(indicator, (tco["Indium tin oxide"]["Location"]["RoW"][indicator] + tco["Indium tin oxide"]["Location"]["RER"][indicator]) / 2) for indicator in indicators]
    )

# what is the cell currently made of?
cell = {
    "Cadmium telluride": {
        "Location": {
            "RoW": {
                "Ecoinvent key": "a5c0a8ffbb9adc293c1d5fde8d6d8b0e",
                "Method LCIA": "Recipe 2016",
                "Unit": "kilogram",
                "Quantity per m2": 0.0434,
                "energy resources: non-renewable, fossil": 8.476115823734938,
                "acidification: terrestrial": 0.005680428600185005,
                "climate change": 0.771485850722027,
                "ecotoxicity: freshwater": 0.24560335706436912,
                "eutrophication: freshwater": 0.0005529781655617064,
                "eutrophication: marine": 3.272302750089776e-05,
                "human toxicity: carcinogenic": 0.08963750989612664,
                "human toxicity: non-carcinogenic": 6.469654842639933,
                "ionising radiation": 0.08143390207352244,
                "land use": 0.02117809395373207,
                "ozone depletion": 3.0870660304249276e-07,
                "particulate matter formation": 0.0024289240188006345,
                "photochemical oxidant formation: human health": 0.0023078967872302743,
                "photochemical oxidant formation: terrestrial ecosystems": 0.002349077381063038,
            },
            "US": {
                "Ecoinvent key": "f040b929c571b35d4aabbc0f7085e1fa",
                "Method LCIA": "Recipe 2016",
                "Unit": "kilogram",
                "Quantity per m2": 0.0434,
                "energy resources: non-renewable, fossil": 7.130239405968148,
                "acidification: terrestrial": 0.004631413094885679,
                "climate change": 0.5907601691821394,
                "ecotoxicity: freshwater": 0.24423033596378285,
                "eutrophication: freshwater": 0.0005320241365843736,
                "eutrophication: marine": 3.133435882155499e-05,
                "human toxicity: carcinogenic": 0.0841762439878008,
                "human toxicity: non-carcinogenic": 6.392129080105372,
                "ionising radiation": 0.13099332340875056,
                "land use": 0.02120771461199264,
                "ozone depletion": 2.47125537040965e-07,
                "particulate matter formation": 0.0020472568865431994,
                "photochemical oxidant formation: human health": 0.0015196531756353406,
                "photochemical oxidant formation: terrestrial ecosystems": 0.0015659360323768137,
            },
        }
    },
    "Cadmium sulfide": {
        "Location": {
            "RoW": {
                "Ecoinvent key": "9bba42b2b5a23c144cac5160614c8999",
                "Method LCIA": "Recipe 2016",
                "Unit": "kilogram",
                "Quantity per m2": 0.0035,
                "energy resources: non-renewable, fossil": 0.2925443286010388,
                "acidification: terrestrial": 0.00013854299152153988,
                "climate change": 0.025551276365487166,
                "ecotoxicity: freshwater": 0.011778787723169655,
                "eutrophication: freshwater": 1.9671440087965658e-05,
                "eutrophication: marine": 1.3318100976465576e-06,
                "human toxicity: carcinogenic": 0.005687415676583242,
                "human toxicity: non-carcinogenic": 0.41660328191637297,
                "ionising radiation": 0.0033460604902129498,
                "land use": 0.0008270333276154631,
                "ozone depletion": 1.2323572414445314e-08,
                "particulate matter formation": 5.7862853123936386e-05,
                "photochemical oxidant formation: human health": 9.678189565206664e-05,
                "photochemical oxidant formation: terrestrial ecosystems": 9.849851057702875e-05,
            },
            "US": {
                "Ecoinvent key": "357aa41c704667095fdab86322a4dde9",
                "Method LCIA": "Recipe 2016",
                "Unit": "kilogram",
                "Quantity per m2": 0.0035,
                "energy resources: non-renewable, fossil": 0.26557298911453453,
                "acidification: terrestrial": 0.00011766036092123133,
                "climate change": 0.022044747626103856,
                "ecotoxicity: freshwater": 0.011760222336886785,
                "eutrophication: freshwater": 1.9266639989213065e-05,
                "eutrophication: marine": 1.2983090286639348e-06,
                "human toxicity: carcinogenic": 0.005574563284042698,
                "human toxicity: non-carcinogenic": 0.41524628864080654,
                "ionising radiation": 0.00424504385269864,
                "land use": 0.000819781372914886,
                "ozone depletion": 1.1208776125712078e-08,
                "particulate matter formation": 5.0443766681301946e-05,
                "photochemical oxidant formation: human health": 8.065104614983476e-05,
                "photochemical oxidant formation: terrestrial ecosystems": 8.243025386636659e-05,
            },
        }
    },
}

# calculate the cell from CdTe and CdS for the US and add the mean TCO values for each indicators
cell_US = dict(
    [(indicator,cell["Cadmium telluride"]["Location"]["US"][indicator] + cell["Cadmium sulfide"]["Location"]["US"][indicator] + tco_mean[indicator]) for indicator in indicators]
    )

# calculate the cell from CdTe and CdS for the RoW and add the mean TCO values for each indicators
cell_RoW = dict(
    [(indicator, cell["Cadmium telluride"]["Location"]["RoW"][indicator] + cell["Cadmium sulfide"]["Location"]["RoW"][indicator] + tco_mean[indicator]) for indicator in indicators]
)

# rebuild the dict
compressed_cell = {"Cell": 
    {"Location": 
        {"US": cell_US, 
         "RoW": cell_Row}
        }
    }

print(compressed_cell)

{'Cell': {'Location': {'RoW': {'Quantity per m2': 0.059800000000000006,
                               'acidification: terrestrial': 0.011462040601639918,
                               'climate change': 1.171101436551921,
                               'ecotoxicity: freshwater': 0.8114555316878436,
                               'energy resources: non-renewable, fossil': 13.016626359757485,
                               'eutrophication: freshwater': 0.0012986981242423606,
                               'eutrophication: marine': 5.975812032428616e-05,
                               'human toxicity: carcinogenic': 0.216971242403287,
                               'human toxicity: non-carcinogenic': 18.780179965327935,
                               'ionising radiation': 0.14257295953046356,
                               'land use': 0.04432739205220329,
                               'ozone depletion': 5.498506685288603e-07,
                               'particulate matter formation'

## Per Device

In [45]:
import intakedata as intake

In [122]:
mono = intake.build_monocrystalline(data.data_dict['Monocrystalline'], intake.raw_layers, intake.raw_indicators)
pprint(mono)

{'cell': {'cell': {'RER': {'Quantity per m2': 0.93,
                           'acidification: terrestrial': 0.6698500318771112,
                           'climate change': 224.9895369726354,
                           'ecotoxicity: freshwater': 12.745151728038394,
                           'energy resources: non-renewable, fossil': 2605.9372614482722,
                           'eutrophication: freshwater': 0.10231620664793152,
                           'eutrophication: marine': 0.01805953918042191,
                           'human toxicity: carcinogenic': 14.583681372245723,
                           'human toxicity: non-carcinogenic': 328.59434788126,
                           'ionising radiation': 24.65667290216485,
                           'land use': 4.385350127443409,
                           'ozone depletion': 8.765482619266396e-05,
                           'particulate matter formation': 0.38462416720011827,
                           'photochemical oxidant formati

In [129]:
poly = intake.build_polycrystalline(data.data_dict['Polycrystalline'], intake.raw_layers, intake.raw_indicators, data.data_dict['Monocrystalline'])
pprint(poly)

{'cell': {'cell': {'RER': {'Quantity per m2': 0.93,
                           'acidification: terrestrial': 0.472200385712758,
                           'climate change': 160.40648427562925,
                           'ecotoxicity: freshwater': 11.439325508098522,
                           'energy resources: non-renewable, fossil': 1885.5145787618267,
                           'eutrophication: freshwater': 0.07717408249372798,
                           'eutrophication: marine': 0.013387909818235889,
                           'human toxicity: carcinogenic': 11.974437221681871,
                           'human toxicity: non-carcinogenic': 286.61691752107953,
                           'ionising radiation': 18.76182641927008,
                           'land use': 3.6355481879430034,
                           'ozone depletion': 6.361869387421675e-05,
                           'particulate matter formation': 0.2606433981961904,
                           'photochemical oxidant for

In [125]:
cigs = intake.build_cigs(data.data_dict['CIGS'], intake.raw_layers, intake.raw_indicators, data.data_dict['Monocrystalline'])
pprint(cigs)

{'cell': {'cell': {'GLO': {'Quantity per m2': 0.0823,
                           'acidification: terrestrial': 0.03204973355769607,
                           'climate change': 4.066335438553314,
                           'ecotoxicity: freshwater': 2.9263855324888626,
                           'energy resources: non-renewable, fossil': 49.73227503055957,
                           'eutrophication: freshwater': 0.00414612897847191,
                           'eutrophication: marine': 0.001276187789852121,
                           'human toxicity: carcinogenic': 0.8530833667969859,
                           'human toxicity: non-carcinogenic': 72.43666191722606,
                           'ionising radiation': 0.42715823878065456,
                           'land use': 0.14793467244745556,
                           'ozone depletion': 2.0201787553640673e-06,
                           'particulate matter formation': 0.011400699160345536,
                           'photochemical oxid

In [124]:
cdte = intake.build_cdte(data.data_dict['CdTe'], intake.raw_layers, intake.raw_indicators, data.data_dict['Monocrystalline'])
pprint(cdte)

{'cell': {'cell': {'RoW': {'Quantity per m2': 0.059800000000000006,
                           'acidification: terrestrial': 0.011462040601639918,
                           'climate change': 1.171101436551921,
                           'ecotoxicity: freshwater': 0.8114555316878436,
                           'energy resources: non-renewable, fossil': 13.016626359757485,
                           'eutrophication: freshwater': 0.0012986981242423606,
                           'eutrophication: marine': 5.975812032428616e-05,
                           'human toxicity: carcinogenic': 0.216971242403287,
                           'human toxicity: non-carcinogenic': 18.780179965327935,
                           'ionising radiation': 0.14257295953046356,
                           'land use': 0.04432739205220329,
                           'ozone depletion': 5.498506685288603e-07,
                           'particulate matter formation': 0.004428452432844908,
                           '

In [130]:
organic = intake.build_organic(data.data_dict['Organic PV'], intake.raw_layers, intake.raw_indicators, data.data_dict['Monocrystalline'])
pprint(organic)

{'cell': {'cell': {'generic': {'Quantity per m2': 0.5359488059999999,
                               'acidification: terrestrial': 0.009516394778590941,
                               'climate change': 2.011714529149483,
                               'ecotoxicity: freshwater': 1.7927172517538252,
                               'energy resources: non-renewable, fossil': 36.88315727186462,
                               'eutrophication: freshwater': 0.0024272983828257846,
                               'eutrophication: marine': 7.938196872967106e-05,
                               'human toxicity: carcinogenic': 0.1922741848464537,
                               'human toxicity: non-carcinogenic': 18.986328116972945,
                               'ionising radiation': 0.08965153809933846,
                               'land use': 0.0866560220253102,
                               'ozone depletion': 5.384256299490329e-06,
                               'particulate matter formation': 0

In [ ]:
perovskite = intake.build_perovskite(data.data_dict['Silicon tandem'], intake.raw_layers, intake.raw_indicators, data.data_dict['Monocrystalline'])
pprint(perovskite)

# Whole Dict

In [162]:
device_dict = {}

for device in intake.raw_devices:
    device_dict[intake.convert_devices[device]] = intake.intake_switcher(device, data.data_dict, intake.raw_layers, intake.raw_indicators)

In [171]:
base_dict = {}

for device in intake.raw_devices:
    clean_device = intake.convert_devices[device]
    base_dict[clean_device] = []
    new_data = device_dict[clean_device]
    
    for l in new_data.keys():
        new_entry = {"title":l,
                     "options":{}}
        
        layer_options = list(new_data[l].keys())
        
        layer_option_locations = {}
        for option in layer_options:
            
            locations = list(new_data[l][option].keys())
            layer_option_locations[option] = locations
        new_entry['options'] = layer_option_locations
        base_dict[clean_device].append(new_entry)

# technology = "Monocrystalline"
pprint(base_dict['cdte'])


[{'options': {'aluminium_profile': ['RER', 'RoW', 'GLO'], 'no_frame': ['none']},
  'title': 'frame'},
 {'options': {'glass': ['RER', 'RoW'],
              'polycarbonate': ['RoW', 'RER'],
              'tempered_glass': ['RoW', 'RER']},
  'title': 'front_cover'},
 {'options': {'cell': ['US', 'RoW']}, 'title': 'cell'},
 {'options': {'epoxy_resin': ['RER', 'RoW'],
              'eva': ['GLO', 'RER', 'RoW'],
              'polyurethane': ['GLO']},
  'title': 'encapsulant'},
 {'options': {'glass': ['RER', 'RoW'],
              'polyethylene_terephthalate': ['RER', 'RoW', 'CA-QC'],
              'polyvinylfluoride_film': ['RoW', 'US'],
              'tempered_glass': ['RoW', 'RER']},
  'title': 'rear_cover'},
 {'options': {'copper': ['RER', 'RoW']}, 'title': 'wires'},
 {'options': {'glass_fibre_reinforced_plastic': ['GLO', 'RER', 'RoW']},
  'title': 'junction_box'},
 {'options': {'aluminium_mounting': ['GLO', 'RER'], 'no_mounting': ['none']},
  'title': 'mounting_system'},
 {'options': {'el

In [175]:
def write_dict_to_json_file(data, file_name):
    """
    Converts a Python dictionary to a JSON-formatted string and writes it to a file.

    :param data: Python dictionary to be converted.
    :param file_name: Name of the file to which the JSON data will be written.
    """
    # Convert the dictionary to a JSON-formatted string
    json_data = json.dumps(data, indent=4)

    # Write the JSON data to a file
    with open(file_name, 'w') as file:
        file.write(json_data)
        
write_dict_to_json_file(device_dict, "/Users/jmccarty/GitHub/parametric-bipv-app/static/data/device_impact_data.json")
write_dict_to_json_file(base_dict, "/Users/jmccarty/GitHub/parametric-bipv-app/static/data/layer_data2.json")

In [1]:
device_dict

NameError: name 'device_dict' is not defined